# Initialization

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml, pandas, logging, random
from pathlib import Path
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
from models.factory import ModelFactory
warnings.filterwarnings('ignore')
with open('../confs/logs.yaml', 'rt') as f:
    config = yaml.safe_load(f.read())
logging.config.dictConfig(config)

In [ ]:
type='player'
X_train, y_train, X_test, y_test, target = get_train_test(train_size=0.8, random_state=42, type=type)
train_scores = get_y('train', type)
test_data = get_X('test', type)

In [ ]:
def eval_model(model, save_model=False, X=X_test):
    start = time.time()
    model.fit()
    score = model.evaluate(X)
    end = time.time()
    logging.debug(f'{model.name}={score} in {numpy.round((end-start), 2)}s')
    if save_model:
        model.save(test_data/{type})
    return {'name': model.name, 'score': score, 'time': numpy.round((end-start), 2)}

In [ ]:
%load_ext autoreload
%autoreload 2
def eval_model_for_name(name, X=X_train, y=y_train):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    factory = ModelFactory(configurations, X, y, train_scores)
    model = factory.get_model(name)
    return eval_model(model)

In [ ]:
def get_features(name = 'ada_boost'):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    factory = ModelFactory(configurations, X_train, y_train, train_scores)
    model = factory.get_model(name)
    eval_model(model)
    df_importances = model.get_feature_importances()
    return list(df_importances['feature'])

In [ ]:
default_features=get_features('ada_boost')
print(default_features)

In [ ]:
def test_features(name = 'xgb_gblinear', save=True, features=default_features):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    if features is None:
        features=default_features
    lines = []
    best_score = 0
    selected_features = []
    for i in range(1, len(features)):
        sub_features = selected_features.copy()
        feature = features[i-1]
        sub_features.append(feature)
        # sub_features = features[:i]
        factory = ModelFactory(configurations, X_train[sub_features], y_train, train_scores)
        model = factory.get_model(name)
        model.name = f'{name}_{i}'
        line = eval_model(model,X=X_test[sub_features])
        line['nbr_features'] = len(sub_features)
        line['model'] = name
        line['feature'] = feature
        line['features'] = sub_features
        lines.append(line)
        if best_score < line['score']:
            selected_features.append(line['feature'])
            best_score = line['score']
    df = pandas.DataFrame(lines)
    if save:
        df.to_csv(f'../data/output/{type}/features/{name}.csv')
    logging.info(f'Features selected for score {name} - {best_score}: {len(selected_features)}, {selected_features}')
    return df

In [ ]:
def plot_features(folder = '../data/output/{type}/features/'):
   import os, pandas
   df = None
   for filename in os.listdir(folder):
      # if not os.path.isfile(filename): continue
      model = filename.split('.')[0]
      df_tmp = pandas.read_csv(f'{folder}{filename}')
      df_tmp[model] = df_tmp['score']
      if df is None:
         df = df_tmp[[model]]
      else:
         df[model] = df_tmp[[model]]
   df.plot.line()

# Feature importances

In [ ]:
name = 'lightgbm'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_GOALS_season_sum', 'PLAYER_KEY_PASSES_season_average', 'PLAYER_CLEARANCES_5_last_match_std', 'PLAYER_BIG_CHANCES_MISSED_season_average', 'PLAYER_BIG_CHANCES_CREATED_5_last_match_std', 'PLAYER_SAVES_INSIDE_BOX_5_last_match_sum', 'PLAYER_GOALS_CONCEDED_5_last_match_sum', 'PLAYER_PENALTIES_MISSES_5_last_match_sum', 'PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_sum', 'PLAYER_TOTAL_DUELS_5_last_match_sum', 'PLAYER_PUNCHES_5_last_match_sum', 'PLAYER_GOALKEEPER_GOALS_CONCEDED_season_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'ada_boost'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_CLEARANCES_season_average', 'PLAYER_GOALS_season_sum', 'PLAYER_KEY_PASSES_season_average', 'PLAYER_SAVES_INSIDE_BOX_season_std', 'PLAYER_DRIBBLED_PAST_5_last_match_std', 'PLAYER_DRIBBLED_ATTEMPTS_5_last_match_average', 'PLAYER_DISPOSSESSED_5_last_match_average', 'PLAYER_DUELS_WON_season_average', 'PLAYER_PENALTIES_COMMITTED_season_average', 'PLAYER_CLEARANCES_season_sum', 'PLAYER_CLEARANCE_OFFLINE_season_sum', 'PLAYER_ACCURATE_CROSSES_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'xgb_gblinear'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_GOALS_season_sum', 'PLAYER_KEY_PASSES_season_average', 'PLAYER_BIG_CHANCES_MISSED_season_average', 'PLAYER_BLOCKED_SHOTS_season_average', 'PLAYER_RATING_5_last_match_average', 'PLAYER_PENALTIES_WON_5_last_match_average', 'PLAYER_GOALKEEPER_GOALS_CONCEDED_5_last_match_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'gradient_boosting'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_GOALS_season_sum', 'PLAYER_BIG_CHANCES_MISSED_season_average', 'PLAYER_ASSISTS_5_last_match_std', 'PLAYER_DISPOSSESSED_season_std', 'PLAYER_SAVES_INSIDE_BOX_5_last_match_sum', 'PLAYER_GOALS_CONCEDED_5_last_match_sum', 'PLAYER_MINUTES_PLAYED_5_last_match_sum', 'PLAYER_SHOTS_BLOCKED_5_last_match_sum', 'PLAYER_REDCARDS_5_last_match_std', 'PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std', 'PLAYER_LONG_BALLS_5_last_match_std', 'PLAYER_SAVES_season_sum', 'PLAYER_SHOTS_BLOCKED_season_average', 'PLAYER_LONG_BALLS_WON_season_std', 'PLAYER_CAPTAIN_5_last_match_sum', 'PLAYER_ASSISTS_season_std', 'PLAYER_STARTING_LINEUP_season_average', 'PLAYER_DUELS_LOST_season_std']
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_GOALS_season_sum', 'PLAYER_BIG_CHANCES_MISSED_season_average', 'PLAYER_ASSISTS_5_last_match_std', 'PLAYER_DISPOSSESSED_season_std', 'PLAYER_SAVES_INSIDE_BOX_5_last_match_sum', 'PLAYER_GOALS_CONCEDED_5_last_match_sum', 'PLAYER_MINUTES_PLAYED_5_last_match_sum', 'PLAYER_SHOTS_BLOCKED_5_last_match_sum', 'PLAYER_REDCARDS_5_last_match_std', 'PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std', 'PLAYER_SAVES_season_sum', 'PLAYER_SHOTS_BLOCKED_season_average', 'PLAYER_LONG_BALLS_WON_season_std', 'PLAYER_CAPTAIN_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'hist_gradient_boosting'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_KEY_PASSES_season_average', 'PLAYER_CLEARANCES_5_last_match_std', 'PLAYER_FOULS_DRAWN_5_last_match_std', 'PLAYER_BIG_CHANCES_CREATED_season_std', 'PLAYER_SHOTS_BLOCKED_5_last_match_std', 'PLAYER_FOULS_5_last_match_average', 'PLAYER_AERIALS_WON_season_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'xgb_classifier'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_YELLOWCARDS_5_last_match_std', 'PLAYER_GOALS_season_sum', 'PLAYER_KEY_PASSES_season_average', 'PLAYER_STARTING_LINEUP_season_sum', 'PLAYER_FOULS_DRAWN_5_last_match_std', 'PLAYER_BIG_CHANCES_MISSED_season_average', 'PLAYER_ASSISTS_5_last_match_std', 'PLAYER_MINUTES_PLAYED_season_std', 'PLAYER_PENALTIES_WON_season_average', 'PLAYER_REDCARDS_season_average', 'PLAYER_KEY_PASSES_5_last_match_sum', 'PLAYER_PENALTIES_SAVED_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'catboost'
features=None 
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_KEY_PASSES_season_average', 'PLAYER_BIG_CHANCES_CREATED_5_last_match_std', 'PLAYER_DISPOSSESSED_season_std', 'PLAYER_BIG_CHANCES_CREATED_season_std', 'PLAYER_FOULS_5_last_match_average', 'PLAYER_FOULS_5_last_match_sum', 'PLAYER_SAVES_5_last_match_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'mlp'
features=None
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_GOALS_CONCEDED_5_last_match_std', 'PLAYER_CLEARANCES_season_average', 'PLAYER_ACCURATE_PASSES_5_last_match_average', 'PLAYER_OFFSIDES_5_last_match_sum', 'PLAYER_SHOTS_BLOCKED_5_last_match_average']
features=['PLAYER_GOALS_CONCEDED_season_average', 'PLAYER_GOALS_CONCEDED_5_last_match_std']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
plot_features(f'../data/output/{type}/features/')

In [ ]:
plot_features(f'../data/output/{type}/all_features/')